# Creating MOCK datasets to make classification and recommendation
Author: Karina Condeixa


In [3]:
# import packages
import pandas as pd
from faker import Faker, providers
from faker.providers.address.de_DE import Provider as DeDeAddressProvider
from faker.generator import random
from faker.providers import BaseProvider
# import random

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# %matplotlib notebook
# %matplotlib 

import folium
# from geopy.geocoders import Nominatim

In [4]:
# define global variables

maximun_n_views = 2001
pct_of_likes_over_views = 20
n_records_ranking = 5000
n_records = 5000

limit = '-30d'  # limit of 30 days of item in the app
maximun_n_hours_avalilable = 100

random_seed = 1000


# post_date: something between the last 30 days - limit = '-30d'
# timer = (date.today()) - post_date

In [5]:
category = ['furniture',  'clothes',  'kitchen_utensils', 'kids', 'other'] 
condition = ['poor', 'acceptable', 'good', 'like_new'] 
available = ['1', '0']    

# postcodes = postcodes_berlin_series
    
n_views = []
for v in range(maximun_n_views): 
    n_views.append(v)

pct_likes = []
for l in range(pct_of_likes_over_views): 
    pct_likes.append(l/100)
 
# datetime post - datetime now, I create a maximum number of hours
n_hours_avalilable = []
for h in range(maximun_n_hours_avalilable):  
    n_hours_avalilable.append(h)
    
distance = []
for d in range(32000):
    distance.append(d)
    


### Postcodes

In [6]:
# immport and clean original dataset removing poostcodes from Germanay, that are not in Berlin
postcodes_de = pd.read_excel(r'data/German-Zip-Codes.xlsx', sheet_name='Berlin')
df = pd.DataFrame(postcodes_de)
df.set_axis(["postcodes_berlin"], axis=1,inplace=True)
df = (df["postcodes_berlin"].str[8:-11])
df.to_csv('data/postcodes_berlin.csv', index=False)

/var/folders/xs/pmxwbppj14b4v76l4llp17n80000gp/T/ipykernel_18710/2575223029.py:4: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(["postcodes_berlin"], axis=1,inplace=True)


### Creating the postcode series

In [7]:
# import postcodes from Berlin and create a dataframe removing indexes and headers
postcodes_berlin = pd.read_csv('data/postcodes_berlin.csv')
print(postcodes_berlin)
postcodes_berlin_series = postcodes_berlin[:][1:].squeeze()

     postcodes_berlin
0               10117
1               10119
2               10178
3               10179
4               10243
..                ...
184             14169
185             14193
186             14195
187             14197
188             14199

[189 rows x 1 columns]


### Creating lists

In [8]:
# Importing Modules
import numpy as np
import random
# Use 'conda install shapely' to import the shapely library.
from shapely.geometry import Polygon, Point
np.random.seed(1000)
random.seed(1000)



### Creating the datasets

In [9]:
# Note: # multi_locale_generator = Faker(['it_IT', 'en_US', 'de-DE', 'pt_BR', 'es-ES', 'fr-FR', 'ru-RU', 'tr-TR'])

# Instantiate Faker with multiple locales
german_locale_generator = Faker(['de_DE'])
fake = Faker()
Faker.seed(0)


### List of latitude and longitude co-ordinates 
Based on a sample of co-ordinates from [free data Berlin](https://daten.berlin.de/datensaetze/stra%C3%9Fenverkehrsunf%C3%A4lle-nach-unfallort-berlin-2021) for Road traffic accidents by accident location in Berlin 2021.


In [10]:
lat_lng_samples = pd.read_csv("data/lat_lng_samples.csv")
lat_lng_samples.head(2)

,lat,lng
0,"5,253,393,955","1,342,689,483"
1,"5,243,995,086","1,339,209,027"


In [11]:
# replace comma to dot
lat_lng_samples = lat_lng_samples.apply(lambda x: x.str.replace(',','.'))

In [12]:
list_lat = lat_lng_samples.lat.values.tolist()
list_lng = lat_lng_samples.lng.values.tolist()
lat_lng_samples.head(2)


,lat,lng
0,5.253.393.955,1.342.689.483
1,5.243.995.086,1.339.209.027


In [13]:
lat_lng_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11197 entries, 0 to 11196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lat     11192 non-null  object
 1   lng     11192 non-null  object
dtypes: object(2)
memory usage: 175.1+ KB


### Item dataset

In [14]:
# An initial classification that is not being used to normalize with UXs categories. But it is saved for future updates.
# category_item = ['furniture-sofa',
#                  'furniture-armchair',
#                  'furniture-chair',
#                  'furniture-table',
#                  'furniture-desk',
#                  'furniture-bed',
#                  'furniture-bookcase',
#                  'furniture-bedside_table',
#                  'furniture-cabinet',
#                  'furniture-wardrobe',
#                  'furniture-shelf',
#                  'furniture-cupboard',
#                  'furniture-rollcontainers',
#                  'furniture-shoe_rack',
#                  'furniture-mirror',
#                  'furniture-cot',
#                  'furniture-trolley',
#                  'appliance-washing_machine',
#                  'appliance-dish_washer',
#                  'appliance-drying_rack',
#                  'appliance-refrigerator',
#                  'appliance-blender',
#                  'appliance-extractor_hood',
#                  'appliance-clothes_iron',
#                  'appliance-vacuum_cleaner',
#                  'appliance-sandwich_maker',
#                  'appliance-kettle',
#                  'appliance-air_conditioner',
#                  'appliance-heater',
#                  'appliance-pan',
#                  'appliance-popcorn_maker',
#                  'appliance-coffee_machine',
#                  'appliance-stove',
#                  'lighting-lighting',
#                  'lighting-chandelier',
#                  'lighting-lightbulb',
#                  'musical_equipment-guitar',
#                  'musical_equipment-sound_amplifier',
#                  'musical_equipment-contrabass',
#                  'musical_equipment-battery',
#                  'musical_equipment-piano',
#                  'tech-desktop',
#                  'tech-laptop',
#                  'tech-phone',
#                  'tech-keyboard',
#                  'clothes-woman_jacket',
#                  'clothes-man_jacket',
#                  'clothes-child_jacket',
#                  'clothes-woman_clothes',
#                  'clothes-man_clothes',
#                  'clothes-child_clothes',
#                  'shoes-woman_shoes',
#                  'shoes-man_shoes',
#                  'shoes-child_shoes',
#                  'miscelaneaous-ironing_board',
#                  'miscelaneaous-picture_frame',
#                  'miscelaneaous-bicycle',
#                  'miscelaneaous-plant',
#                  'miscelaneaous-carpet',
#                  'miscelaneaous-roller_skates',
#                  'miscelaneaous-ski_skates',
#                  'miscelaneaous-books',
#                  'miscelaneaous-purse',
#                  'miscelaneaous-suitcase',
#                  'miscelaneaous-shopping_venture',
#                  'miscelaneaous-board',
#                  'miscelaneaous-frame',
#                  'home-mattress', 
#                  'home-carpet',
#                  'kids-stroller',
#                  'kids-baby_carriage']

In [15]:
# addresses = []
# for _ in range(50000000):
#     address = german_locale_generator.address()
#     if 'Berlin' in address:
#         addresses.append(address)
#     else:
#         if len(addresses) < 1500: 
#             continue
#         else: 
#             break
            
# df['is_berlin'] = df['address'].apply(lambda x: 1 if 'Berlin' in x else 0)

In [16]:

#         datetime_iteration1 = fake.date_between_dates(limit,'now')
#         datetime_iteration2 = fake.date_between_dates(limit,'now')
#         if datetime_iteration1 <= datetime_iteration2:
#             item[i]['item_timer'] = datetime_iteration1
#             item[i]['item_timer'] = datetime_iteration2
#         else:
#             item[i]['item_timer'] = datetime_iteration2
#             item[i]['item_timer'] = datetime_iteration1 

#         item[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now')

In [17]:
# # Define the desired polygon : points choosen in Google maps

# poly = Polygon([(52.645883, 13.395869), 
#                 (52.526568, 13.645808),
#                 (52.381789, 13.405482),
#                 (52.484773, 13.136317)])


# min_x = 52.381789
# max_x = 52.645883
# min_y = 13.136317
# max_y = 13.645808

# # Defining the randomization generator
# def polygon_random_points (poly, num_records):
#     min_x, min_y, max_x, max_y = poly.bounds
#     points = []
#     while len(points) < num_records:
#         random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
#         if (random_point.within(poly)):
#             points.append(random_point)
#     return points
    # Choose the number of points desired. T\ 
# points = polygon_random_points(poly,n_records)
# # Testing the results.
# for p in points:
#     print(p.x,",",p.y)


In [18]:
# from math import sin, cos, sqrt, atan2, radians

# # Approximate radius of earth in km
# R = 6373.0

# lat1 = radians(52.2296756)
# lon1 = radians(21.0122287)
# lat2 = radians(52.406374)
# lon2 = radians(16.9251681)

# dlon = lon2 - lon1
# dlat = lat2 - lat1

# a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
# c = 2 * atan2(sqrt(a), sqrt(1 - a))

# distance = R * c

# print("Result: ", distance)
# print("Should be: ", 278.546, "km")

In [19]:

def data_classification_1(num_records): 
  
    # dictionary 
    item ={} 
    for i in range(0, num_records): 
        item[i] = {}
#         item[i]['item_status'] = 1
        item[i]['item_category'] = np.random.choice(category)  
        item[i]['item_condition'] = np.random.choice(condition)

#         item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable) # in 30 days
        
#         datetime_iteration1 = fake.date_between_dates(limit,'now')
#         datetime_iteration2 = fake.date_between_dates(limit,'now')
#         if datetime_iteration1 <= datetime_iteration2:
#             item[i]['item_timer'] = datetime_iteration1
#             item[i]['item_timer'] = datetime_iteration2
#         else:
#             item[i]['item_timer'] = datetime_iteration2
#             item[i]['item_timer'] = datetime_iteration1 

        item[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
    
        item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now')  # in days, can be improved to hours
        item[i]['distance'] = np.random.choice(distance)
        
 # This date shold be later than the post
        item[i]['n_views'] = np.random.choice(n_views)
        item[i]['n_likes'] = int(item[i]['n_views'] * (np.random.choice(pct_likes)))
        
        item[i]['item_available'] = np.random.choice(available)
             
    return item

In [20]:
ml_class_df = pd.DataFrame(data_classification_1(n_records)).transpose()
ml_class_df.head()

,item_category,item_condition,item_postcode,item_available_timer,distance,n_views,n_likes,item_available
0,kids,like_new,12109,64,15611,350,3,0
1,furniture,acceptable,10789,40,5993,348,13,1
2,kitchen_utensils,good,13627,20,27122,1310,117,1
3,kitchen_utensils,good,13505,30,10484,205,36,1
4,kitchen_utensils,good,10405,61,11845,1740,191,1


In [21]:
# ml_class_df['item_id'] = ml_class_df.index +1  # add item_id
# item_id_series = ml_class_df['item_id']  # storage in a variable to use later

# ml_class_df.head(10)

In [22]:
# def search(num_records): 
  
#     # dictionary 
#     search ={} 
#     for i in range(0, num_records): 
#         search[i] = {}
# #         item[i]['item_status'] = 1
#         search[i]['item_category'] = np.random.choice(category)
#         search[i]['item_id'] = np.random.choice(range(1, n_records))  
# #         search[i]['local'] = np.random.choice(XXXXX)
# #         search[i]['time'] = np.random.choice(xxx)

#     return search

In [23]:
### User

In [24]:
# user_df = item_df.copy()

In [25]:
# user_df = user_df.rename(columns={"item_id": "user_id"})

In [26]:
# user_df.head(10)

In [27]:
# # item_df.get('item_timer')

# item_df['item_timer'] = item_df['item_timer'].astype(str).str[:2].str.strip()
# item_df['item_timer'] = item_df['item_timer'].astype(int)

# item_df = item_df.rename(columns={'item_timer': 'item_timer_days'})

# # item_df.drop('item_timer', axis=1)
# item_df.info()

# item_df.head()

In [28]:
# # saves csv files
# ml_class_df.to_csv('data/ml_class_df.csv', index=False)
# # user_df.to_csv('data/user_df.csv', index=False)

In [29]:
def create_data_rankings(num_records_ranking): 
  
    # dictionary 
    item ={} 
    for i in range(0, num_records_ranking): 
        item[i] = {}
#         item[i]['item_status'] = 1
        item[i]['item_category'] = np.random.choice(category)
        item[i]['item_condition'] = np.random.choice(condition)

        item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now') # in days, can be improved to 
        
        item[i]['distance'] = np.random.choice(distance)
        
        
 
 # This date shold be later than the post
        item[i]['n_views'] = np.random.choice(n_views)
        item[i]['n_likes'] = int(item[i]['n_views'] * (np.random.choice(pct_likes)))
              
    return item

In [30]:
ranking_df = pd.DataFrame(create_data_rankings(n_records_ranking)).transpose()
ranking_df.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes
0,other,poor,66,22768,1349,94
1,other,poor,94,8623,23,1
2,other,acceptable,31,31479,146,13
3,furniture,acceptable,36,15871,1671,0
4,other,poor,9,21883,1878,281


In [31]:
ranking_df['item_id'] = ranking_df.index +1  # add item_id
item_id_series = ranking_df['item_id']  # storage in a variable to use later

ranking_df.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_id
0,other,poor,66,22768,1349,94,1
1,other,poor,94,8623,23,1,2
2,other,acceptable,31,31479,146,13,3
3,furniture,acceptable,36,15871,1671,0,4
4,other,poor,9,21883,1878,281,5


In [32]:
# ranking_df['item_condition']
# ['poor', 'acceptable', 'Good', 'like_new'] 

ranking_df['item_condition'] = ranking_df.item_condition.replace(['like_new', 'good', 'acceptable', 'poor'], [1, 2, 3, 4])

ranking_df.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_id
0,other,4,66,22768,1349,94,1
1,other,4,94,8623,23,1,2
2,other,3,31,31479,146,13,3
3,furniture,3,36,15871,1671,0,4
4,other,4,9,21883,1878,281,5


In [33]:
ranking_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   item_category         5000 non-null   object
 1   item_condition        5000 non-null   int64 
 2   item_available_timer  5000 non-null   object
 3   distance              5000 non-null   object
 4   n_views               5000 non-null   object
 5   n_likes               5000 non-null   object
 6   item_id               5000 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 312.5+ KB


In [34]:
# saves csv files
ranking_df.to_csv('../recommendation/data/ranking_df.csv', index=False)

In [35]:
ml_class_df.to_csv('data/ml_class_df.csv', index=False)

In [36]:
ml_class_df.head()

,item_category,item_condition,item_postcode,item_available_timer,distance,n_views,n_likes,item_available
0,kids,like_new,12109,64,15611,350,3,0
1,furniture,acceptable,10789,40,5993,348,13,1
2,kitchen_utensils,good,13627,20,27122,1310,117,1
3,kitchen_utensils,good,13505,30,10484,205,36,1
4,kitchen_utensils,good,10405,61,11845,1740,191,1


In [37]:
encoded = pd.read_csv('data/item_data_encoded.csv',index_col=None)
encoded.head(14)

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_available
0,1,4,13,26541,803,48,0
1,4,1,75,14931,211,40,1
2,1,1,33,28823,1521,288,0
3,4,4,36,23307,1968,196,0
4,3,1,17,17672,294,0,0
5,4,3,61,3661,913,118,1
6,2,1,44,15960,693,83,0
7,1,3,10,12569,1328,106,0
8,2,4,50,25407,693,20,1
9,2,4,78,2285,293,11,0


In [51]:
# review later, I used a file made in excel

n_records_new = 1500

def new_data(n_records_new): 
  
    # dictionary 
    new_data ={} 
    for i in range(0, n_records_new): 
        new_data[i] = {}
#         item[i]['item_status'] = 1
        new_data[i]['item_category'] = np.random.choice(category)
#         new_data[i]['item_id'] = np.random.choice(range(1, n_records_new))  
        new_data[i]['item_condition'] = np.random.choice(condition)

#         item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable) # in 30 days
        
#         datetime_iteration1 = fake.date_between_dates(limit,'now')
#         datetime_iteration2 = fake.date_between_dates(limit,'now')
#         if datetime_iteration1 <= datetime_iteration2:
#             item[i]['item_timer'] = datetime_iteration1
#             item[i]['item_timer'] = datetime_iteration2
#         else:
#             item[i]['item_timer'] = datetime_iteration2
#             item[i]['item_timer'] = datetime_iteration1 

#         new_data[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
    
        new_data[i]['item_available_timer'] = np.random.choice(n_hours_avalilable)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now')  # in days, can be improved to hours
        new_data[i]['distance'] = np.random.choice(distance)
        
 # This date shold be later than the post
        new_data[i]['n_views'] = np.random.choice(n_views)
        new_data[i]['n_likes'] = int(new_data[i]['n_views'] * (np.random.choice(pct_likes)))
        new_data[i]['item_available'] = np.random.choice(available)
        
             
    return new_data

new_data(n_records_new)

{0: {'item_category': 'other',
  'item_condition': 'good',
  'item_available_timer': 34,
  'distance': 23127,
  'n_views': 489,
  'n_likes': 68,
  'item_available': '1'},
 1: {'item_category': 'clothes',
  'item_condition': 'poor',
  'item_available_timer': 84,
  'distance': 17809,
  'n_views': 253,
  'n_likes': 45,
  'item_available': '1'},
 2: {'item_category': 'furniture',
  'item_condition': 'like_new',
  'item_available_timer': 30,
  'distance': 6492,
  'n_views': 755,
  'n_likes': 128,
  'item_available': '1'},
 3: {'item_category': 'furniture',
  'item_condition': 'like_new',
  'item_available_timer': 76,
  'distance': 25290,
  'n_views': 1527,
  'n_likes': 122,
  'item_available': '0'},
 4: {'item_category': 'furniture',
  'item_condition': 'acceptable',
  'item_available_timer': 61,
  'distance': 2017,
  'n_views': 1575,
  'n_likes': 204,
  'item_available': '1'},
 5: {'item_category': 'kids',
  'item_condition': 'good',
  'item_available_timer': 2,
  'distance': 28113,
  'n_v

In [52]:
new_data= pd.DataFrame(new_data(n_records_new)).transpose()
new_data.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_available
0,other,like_new,89,7034,356,14,0
1,clothes,good,68,30304,941,37,1
2,clothes,good,64,1373,1627,32,0
3,other,acceptable,33,7584,695,6,0
4,kitchen_utensils,acceptable,52,22752,116,19,1


In [53]:
new_data.head()

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_available
0,other,like_new,89,7034,356,14,0
1,clothes,good,68,30304,941,37,1
2,clothes,good,64,1373,1627,32,0
3,other,acceptable,33,7584,695,6,0
4,kitchen_utensils,acceptable,52,22752,116,19,1


In [54]:
new_data.to_csv('data/new_data.csv', index=False)

In [55]:
# new_data.to_csv('data/new_data.csv', index=False) #

### Dataset to test recommendation

In [156]:
# title = ['kids', 'other', ['kitchen_utensils', 'kids'], ['furniture',  'clothes',  'kitchen_utensils'], ['kitchen_utensils', 'kids'], ['clothes', 'kids', 'other']]


# get_first_value = lambda x: x[0] if type(x) == list else x
# print(title)

In [185]:
# def get_random_title():
#     random_title = np.random.choice(title)
#     if isinstance(random_title, list):
#         return random_title
#     else:
#         return [random_title]
# get_random_title()

/var/folders/xs/pmxwbppj14b4v76l4llp17n80000gp/T/ipykernel_7648/1913666005.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  random_title = np.random.choice(title)


['Furniture', 'Clothes', 'Kitchen']

In [1]:
import numpy as np
import pandas as pd
from faker import Faker, providers
from faker.providers.address.de_DE import Provider as DeDeAddressProvider
from faker.generator import random
from faker.providers import BaseProvider
import random

maximun_n_views = 2001
np.random.seed(123) 


n_views = []
for v in range(maximun_n_views):  
    n_views.append(v)

title = ['Furniture',  'Clothes', 'All', ['Plants', 'Kitchen', 'Kids toys'], 'All', ['Kitchen', 'Kids toys'], ['Furniture',  'Clothes',  'Kitchen'], ['Kitchen', 'Kids toys'], ['Clothes', 'Kids toys', 'Plants'], 'All']
description = ['anything', 'everything', 'whaterver', 'hellooo', 'testing', 'bug', 'solving the problem', 'what day is today?', 'today is the presentation day', 'we are the second team to present', 'uhuuuu', 'yeyyyy']
category = ['furniture',  'clothes',  'kitchen_utensils', 'kids', 'other'] 
condition = ['poor', 'acceptable', 'good', 'like_new'] 
available = ['1', '0'] 

num_likes_reco = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,24,15,16,17,18,19,20]

limit = '-30d'

def get_random_title():
    random_title = np.random.choice(title)
    if isinstance(random_title, list):
        return random_title
    else:
        return [random_title]
get_random_title()

def data_recommendation(num_reco_records): 
  
    # dictionary 
    item ={} 
    for i in range(0, num_reco_records): 
        item[i] = {}
        fake = Faker()
        item[i]['title']= get_random_title()  
        item[i]['description']= np.random.choice(description) 
        item[i]['available'] = np.random.choice(available) 
        item[i]['condition'] = np.random.choice(condition)
        item[i]['timedate_creation'] = fake.date_between_dates(limit,'now')
        item[i]['latitude'] = float(fake.latitude())
        item[i]['longitude'] = float(fake.longitude())
        item[i]['categories'] = np.random.choice(category)
        item[i]['views'] = np.random.choice(n_views)
        item[i]['likes'] = np.random.choice(num_likes_reco) # not dependent from view, only to test reco
        item[i]['point'] =  [f"SRID=4326;POINT ({fake.longitude()} {fake.latitude()})"]
    
    return item 

num_reco_records = 100        
data_recommendation(num_reco_records) 
     

/var/folders/xs/pmxwbppj14b4v76l4llp17n80000gp/T/ipykernel_18710/1103110266.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  random_title = np.random.choice(title)


{0: {'title': ['All'],
  'description': 'solving the problem',
  'available': '0',
  'condition': 'like_new',
  'timedate_creation': datetime.date(2023, 3, 23),
  'latitude': -1.0890235,
  'longitude': 14.192423,
  'categories': 'kitchen_utensils',
  'views': 1147,
  'likes': 2,
  'point': ['SRID=4326;POINT (75.829689 -42.251942)']},
 1: {'title': ['Furniture'],
  'description': 'everything',
  'available': '1',
  'condition': 'like_new',
  'timedate_creation': datetime.date(2023, 3, 24),
  'latitude': 64.2027365,
  'longitude': -48.204128,
  'categories': 'clothes',
  'views': 1568,
  'likes': 15,
  'point': ['SRID=4326;POINT (-131.665394 -24.793419)']},
 2: {'title': ['Furniture'],
  'description': 'we are the second team to present',
  'available': '0',
  'condition': 'good',
  'timedate_creation': datetime.date(2023, 3, 11),
  'latitude': -45.7367635,
  'longitude': -41.254889,
  'categories': 'other',
  'views': 608,
  'likes': 17,
  'point': ['SRID=4326;POINT (-148.752235 -32.969

In [41]:
reco_df= pd.DataFrame(data_recommendation(num_reco_records)).transpose()
reco_df.head()

/var/folders/xs/pmxwbppj14b4v76l4llp17n80000gp/T/ipykernel_18710/1103110266.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  random_title = np.random.choice(title)


,title,description,available,condition,timedate_creation,latitude,longitude,categories,views,likes,point
0,[Furniture],anything,0,good,2023-03-28,22.896325,-158.265114,furniture,425,3,[SRID=4326;POINT (-40.995129 47.244262)]
1,"[Clothes, Kids toys, Plants]",what day is today?,0,good,2023-04-02,18.69868,-17.160223,other,826,24,[SRID=4326;POINT (75.871040 6.1131465)]
2,[Furniture],we are the second team to present,1,good,2023-04-06,-31.362226,90.951694,other,1915,5,[SRID=4326;POINT (-105.227997 -14.344729)]
3,[Furniture],testing,0,acceptable,2023-03-16,-64.544062,151.984327,kitchen_utensils,1208,1,[SRID=4326;POINT (-45.510434 52.958961)]
4,[Clothes],uhuuuu,1,poor,2023-04-07,-50.55035,-13.490789,other,266,4,[SRID=4326;POINT (-126.976835 -70.204917)]


In [42]:
reco_df.head(2)
# reco_df.info()

,title,description,available,condition,timedate_creation,latitude,longitude,categories,views,likes,point
0,[Furniture],anything,0,good,2023-03-28,22.896325,-158.265114,furniture,425,3,[SRID=4326;POINT (-40.995129 47.244262)]
1,"[Clothes, Kids toys, Plants]",what day is today?,0,good,2023-04-02,18.69868,-17.160223,other,826,24,[SRID=4326;POINT (75.871040 6.1131465)]


In [43]:
reco_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   title              100 non-null    object
 1   description        100 non-null    object
 2   available          100 non-null    object
 3   condition          100 non-null    object
 4   timedate_creation  100 non-null    object
 5   latitude           100 non-null    object
 6   longitude          100 non-null    object
 7   categories         100 non-null    object
 8   views              100 non-null    object
 9   likes              100 non-null    object
 10  point              100 non-null    object
dtypes: object(11)
memory usage: 9.4+ KB


In [44]:
reco_df['category_ml'] = reco_df['title'].apply(lambda x: x[0])

reco_df['id'] = reco_df.index +1  # add item_id
reco_series = reco_df['id']  # storage in a variable to use later


reco_df['timedate_creation'] = pd.to_datetime(reco_df['timedate_creation'])

reco_df['timedate_creation'] = reco_df['timedate_creation'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]


reco_df.head()

,title,description,available,condition,timedate_creation,latitude,longitude,categories,views,likes,point,category_ml,id
0,[Furniture],anything,0,good,2023-03-28 00:00:00.000000,22.896325,-158.265114,furniture,425,3,[SRID=4326;POINT (-40.995129 47.244262)],Furniture,1
1,"[Clothes, Kids toys, Plants]",what day is today?,0,good,2023-04-02 00:00:00.000000,18.69868,-17.160223,other,826,24,[SRID=4326;POINT (75.871040 6.1131465)],Clothes,2
2,[Furniture],we are the second team to present,1,good,2023-04-06 00:00:00.000000,-31.362226,90.951694,other,1915,5,[SRID=4326;POINT (-105.227997 -14.344729)],Furniture,3
3,[Furniture],testing,0,acceptable,2023-03-16 00:00:00.000000,-64.544062,151.984327,kitchen_utensils,1208,1,[SRID=4326;POINT (-45.510434 52.958961)],Furniture,4
4,[Clothes],uhuuuu,1,poor,2023-04-07 00:00:00.000000,-50.55035,-13.490789,other,266,4,[SRID=4326;POINT (-126.976835 -70.204917)],Clothes,5


In [45]:
# Convert single column to int dtype.
cols_to_int = ['id', 'available', 'views', 'likes']
cols_to_float = ['latitude', 'longitude']

reco_df[cols_to_int] = reco_df[cols_to_int].astype('int')
reco_df[cols_to_float] = reco_df[cols_to_float].astype('float')

reco_df.info()
# df['Fee'] = df['Fee'].astype('int')
# df['points'] = df['points'].astype(float)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              100 non-null    object 
 1   description        100 non-null    object 
 2   available          100 non-null    int64  
 3   condition          100 non-null    object 
 4   timedate_creation  97 non-null     object 
 5   latitude           100 non-null    float64
 6   longitude          100 non-null    float64
 7   categories         100 non-null    object 
 8   views              100 non-null    int64  
 9   likes              100 non-null    int64  
 10  point              100 non-null    object 
 11  category_ml        100 non-null    object 
 12  id                 100 non-null    int64  
dtypes: float64(2), int64(4), object(7)
memory usage: 15.0+ KB


In [46]:
# reco_df.head()

In [47]:
reco_df = reco_df[['id', 'title', 'description', 'available', 'timedate_creation',
       'latitude', 'longitude', 'condition', 'categories', 'category_ml',
       'views', 'likes', 'point']]

In [48]:
reco_df.head()

,id,title,description,available,timedate_creation,latitude,longitude,condition,categories,category_ml,views,likes,point
0,1,[Furniture],anything,0,2023-03-28 00:00:00.000000,22.896325,-158.265114,good,furniture,Furniture,425,3,[SRID=4326;POINT (-40.995129 47.244262)]
1,2,"[Clothes, Kids toys, Plants]",what day is today?,0,2023-04-02 00:00:00.000000,18.698680,-17.160223,good,other,Clothes,826,24,[SRID=4326;POINT (75.871040 6.1131465)]
2,3,[Furniture],we are the second team to present,1,2023-04-06 00:00:00.000000,-31.362226,90.951694,good,other,Furniture,1915,5,[SRID=4326;POINT (-105.227997 -14.344729)]
3,4,[Furniture],testing,0,2023-03-16 00:00:00.000000,-64.544062,151.984327,acceptable,kitchen_utensils,Furniture,1208,1,[SRID=4326;POINT (-45.510434 52.958961)]
4,5,[Clothes],uhuuuu,1,2023-04-07 00:00:00.000000,-50.550350,-13.490789,poor,other,Clothes,266,4,[SRID=4326;POINT (-126.976835 -70.204917)]


In [49]:
reco_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 100 non-null    int64  
 1   title              100 non-null    object 
 2   description        100 non-null    object 
 3   available          100 non-null    int64  
 4   timedate_creation  97 non-null     object 
 5   latitude           100 non-null    float64
 6   longitude          100 non-null    float64
 7   condition          100 non-null    object 
 8   categories         100 non-null    object 
 9   category_ml        100 non-null    object 
 10  views              100 non-null    int64  
 11  likes              100 non-null    int64  
 12  point              100 non-null    object 
dtypes: float64(2), int64(4), object(7)
memory usage: 15.0+ KB


In [196]:
reco_df.to_csv('../recommendation/data/mock_data_reco.csv', index=False) # I had an error and made in excel